In [1]:
import pandas as pd
import numpy as np
from pymatgen.core import Composition
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
import math
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
config = {
    "mathtext.fontset":'stix',
    "font.family":'serif',
    "font.serif": ['Arial'],
    "font.size": 24,
    'axes.unicode_minus': False 
}
plt.rcParams.update(config)
plt.rcParams['axes.unicode_minus'] = False  
large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (8, 6),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.rcParams['figure.dpi'] = 400
plt.rcParams['axes.grid'] = False

In [2]:
def init_population(n,c):
    return np.array([[math.ceil(e) for e in pop] for pop in (np.random.rand(n,c)-0.5)]), np.zeros((2,c))-1

def single_point_crossover(population):
    r,c, n = population.shape[0], population.shape[1], np.random.randint(1,population.shape[1])         
    for i in range(0,r,2):                
        population[i], population[i+1] = np.append(population[i][0:n],population[i+1][n:c]),np.append(population[i+1][0:n],population[i][n:c])        
    return population

def flip_mutation(population):
    return population.max() - population

def random_selection(population):
    r = population.shape[0]
    new_population = population.copy()    
    for i in range(r):        
        new_population[i] = population[np.random.randint(0,r)]
    return new_population
def predictive_model(X,y,model):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True, random_state=42)
    mdl = model
    mdl.fit(X_train,y_train) 
    y_pred = mdl.predict(X_test)
    return r2_score(y_test, y_pred)
def get_fitness(data,feature_list,target,population,model):    
    fitness = []
    for i in range(population.shape[0]):    
        columns = [feature_list[j] for j in range(population.shape[1]) if population[i,j]==1]
        score = predictive_model(data[columns], data[target],model)
        fitness.append(score)                
    return fitness
def memorize(pop, memory):      
    return np.append(memory, pop.reshape(1,memory.shape[1]), axis=0)
def replace_duplicate(population, memory):    
    for i in range(population.shape[0]):         
        counter = 0                
        while any(np.all(memory == population[i], axis=1)) and (counter < 100): 
            population[i] = np.array([math.ceil(k) for k in (np.random.rand(population.shape[1])-0.5)])                    
            counter += 1                    
        memory = memorize(population[i], memory)   
    return population, memory

In [3]:
def ga(data, feature_list, target, n, max_iter,model):
    
    c = len(feature_list)
    
    population, memory = init_population(n,c)
    population, memory = replace_duplicate(population, memory)    
    
    fitness = get_fitness(data, feature_list, target, population,model)    
    
    optimal_value = max(fitness)
    optimal_solution = population[np.where(fitness==optimal_value)][0]    
    
    for i in range(max_iter): 
        start = time()
        population = random_selection(population)
        population = single_point_crossover(population)                        
        if np.random.rand() < 0.3:
            population = flip_mutation(population)   
        
        population, memory = replace_duplicate(population, memory)
                
        fitness = get_fitness(data, feature_list, target, population,model)
                
        if max(fitness) > optimal_value:
            optimal_value    = max(fitness)
            optimal_solution = population[np.where(fitness==optimal_value)][0]                               
        end = time()
        print(f"第 %d 次迭代T = %0.4f, optimize_value = %0.4f" % (i,end - start,optimal_value))
    return optimal_solution, optimal_value

In [ ]:
small_data = pd.read_csv('/data/single_perovskite_oxide.csv')
small_data = small_data.drop(columns = 'Unnamed: 0').reset_index(drop=True)
small_data = small_data.drop(columns = 'composition').reset_index(drop=True)
small_data['composition'] = small_data['formula'].map(Composition)
y_eg_abo3 = small_data['target']

In [ ]:
re_data = pd.read_csv('/data/all_features_df.csv')
ABO3_atomic_features = pd.read_csv('/data/small_atomic_features_new.csv').drop(columns = 'Unnamed: 0')
atomic_features = pd.read_csv('/data/ao_features.csv')

In [6]:
ABO3_atomic_features

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_p_neg_1,...,X_atomic_r_val_1,X_X,X_atomic_ie_1,X_atomic_ie_2,X_atomic_npunf,X_atomic_npval,X_atomic_nsval,X_atomic_nunf,X_atomic_nval,X_atomic_pol
0,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
1,-4.38669,-2.25963,-2.25963,-2.25963,1.578750,6.198956,1.68613,7.08536,1.84895,0.64805,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
2,-2.01997,-2.01997,-0.47011,-0.47011,0.846499,4.172046,2.68397,6.53600,1.94775,1.37595,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
3,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
4,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
5,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
6,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
7,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
8,-3.55283,-1.85729,-1.85729,-1.85729,-2.664785,2.129405,0.60005,6.41133,2.09365,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802
9,-2.78148,-2.78148,-1.07232,-1.07232,2.903783,6.083156,1.87232,6.37620,2.11275,0.00000,...,0.41365,3.44,13.61805,35.1211,2.0,4.0,2.0,2.0,6.0,0.802


In [7]:
atomic_labels = atomic_features.columns
select_atomic_labels = atomic_labels
atomic_features

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,B_atomic_ie_1,B_atomic_ie_2,B_atomic_ndunf,B_atomic_ndval,B_atomic_npunf,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi,B_atomic_pol
0,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,6.766510,16.485700,5.0,5.0,0.0,0.0,6.0,6.0,4.650,11.600
1,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.881010,17.084000,3.0,7.0,0.0,0.0,3.0,9.0,5.100,7.500
2,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.902400,16.187700,4.0,6.0,0.0,0.0,4.0,8.0,4.930,8.400
3,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,6.758850,14.000000,6.0,4.0,0.0,0.0,7.0,5.0,4.050,15.700
4,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.343920,14.632200,0.0,10.0,4.0,2.0,4.0,14.0,4.150,7.060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72843,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,5.925526,11.667650,8.7,1.3,0.0,0.0,17.8,4.0,3.366,25.100
72844,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,5.676880,13.749545,6.3,3.7,1.5,0.3,16.9,6.7,3.456,23.156
72845,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,6.322594,12.498752,6.3,0.7,1.2,0.6,16.6,4.0,3.636,22.379
72846,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,6.141961,0.000000,8.4,1.6,0.0,0.0,17.5,8.5,3.441,24.650


# EC

In [8]:
data = re_data[select_atomic_labels].copy(deep = True)
data

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,B_atomic_ie_1,B_atomic_ie_2,B_atomic_ndunf,B_atomic_ndval,B_atomic_npunf,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi,B_atomic_pol
0,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,6.766510,16.485700,5.0,5.0,0.0,0.0,6.0,6.0,4.650,11.600
1,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.902400,16.187700,4.0,6.0,0.0,0.0,4.0,8.0,4.930,8.400
2,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,6.633900,13.100000,8.0,2.0,0.0,0.0,8.0,4.0,3.450,17.900
3,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,7.899430,15.934610,0.0,10.0,4.0,2.0,4.0,14.0,4.550,5.840
4,-4.210348,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,1.754248,6.900584,1.90799,0.81939,...,6.825070,15.000000,8.0,2.0,0.0,0.0,8.0,18.0,3.600,16.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,5.867260,11.525000,8.7,1.3,0.0,0.0,17.8,4.0,3.261,26.090
35321,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,6.673900,13.058342,6.3,0.7,0.0,0.0,15.4,3.4,3.741,22.400
35322,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,6.246919,12.375383,6.3,3.7,1.2,0.6,16.6,7.0,3.591,22.472
35323,-3.709160,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.136460,5.964720,2.16695,2.26615,...,5.924611,12.095000,8.7,1.3,0.0,0.0,17.8,8.2,3.306,25.580


In [11]:
from scipy.stats import spearmanr
def remove_collinear_features(x, y,threshold):
    '''
    Objective:
        Remove collinear features in a dataframe with a correlation coefficient
        greater than the threshold. Removing collinear features can help a model
        to generalize and improves the interpretability of the model.
        
    Inputs: 
        threshold: any features with correlations greater than this value are removed
    
    Output: 
        dataframe that contains only the non-highly-collinear features
    '''  
    # Calculate the correlation matrix
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterate through the correlation matrix and compare correlations
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            
            # If correlation exceeds the threshold
            if val >= threshold:
                # Print the correlated features and the correlation value
#                 print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                x1 = x[col.values[0]]
                x2 = x[row.values[0]]
                corr1,p1 = spearmanr(x1, y)
                corr2,p2 = spearmanr(x2, y)
                if np.abs(corr1) < np.abs(corr2):
                    drop_cols.append(col.values[0])
                else:
                    drop_cols.append(row.values[0])
    # Drop one of each pair of correlated columns
    drops = set(drop_cols)
    x = x.drop(columns = drops)
    return x

In [12]:
X = data.copy(deep = True)
y = re_data['Eg']
X = remove_collinear_features(X,y,0.8)
fl = X.columns
X = data[fl]
X

,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_r_val,A_atomic_r_s_1,A_atomic_r_d_1,A_atomic_r_val_1,A_atomic_ie_2,...,B_atomic_r_val_neg_1,B_atomic_r_val,B_atomic_r_val_05,B_atomic_r_d_1,B_atomic_ie_2,B_atomic_ndval,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi
0,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.45805,0.44005,1.41305,0.44335,16.485700,5.0,0.0,6.0,6.0,4.650
1,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.38335,0.38735,0.39065,0.38895,16.187700,6.0,0.0,4.0,8.0,4.930
2,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.90925,0.85375,0.83435,0.82115,13.100000,2.0,0.0,8.0,4.0,3.450
3,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,1.32395,1.18975,1.14245,0.26635,15.934610,10.0,2.0,4.0,14.0,4.550
4,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.98135,0.90685,0.88435,0.86755,15.000000,2.0,0.0,8.0,18.0,3.600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.20619,1.12409,1.64012,1.06461,11.525000,1.3,0.0,17.8,4.0,3.261
35321,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.45087,1.19216,1.70297,0.00000,13.058342,0.7,0.0,15.4,3.4,3.741
35322,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.33060,1.22489,1.73255,0.89817,12.375383,3.7,0.6,16.6,7.0,3.591
35323,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.22782,1.14002,1.65512,1.07853,12.095000,1.3,0.0,17.8,8.2,3.306


In [13]:
X_GA = X.copy(deep = True)
X_GA['Eg'] = y
df = X_GA
df

,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_r_val,A_atomic_r_s_1,A_atomic_r_d_1,A_atomic_r_val_1,A_atomic_ie_2,...,B_atomic_r_val,B_atomic_r_val_05,B_atomic_r_d_1,B_atomic_ie_2,B_atomic_ndval,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi,Eg
0,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.44005,1.41305,0.44335,16.485700,5.0,0.0,6.0,6.0,4.650,2.146025
1,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.38735,0.39065,0.38895,16.187700,6.0,0.0,4.0,8.0,4.930,1.374682
2,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.85375,0.83435,0.82115,13.100000,2.0,0.0,8.0,4.0,3.450,4.177025
3,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,1.18975,1.14245,0.26635,15.934610,10.0,2.0,4.0,14.0,4.550,3.941249
4,-2.165658,-2.165658,-2.165658,-0.299291,4.230849,0.53737,0.00000,0.0,1.11809,12.453816,...,0.90685,0.88435,0.86755,15.000000,2.0,0.0,8.0,18.0,3.600,4.529786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35320,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.12409,1.64012,1.06461,11.525000,1.3,0.0,17.8,4.0,3.261,3.843865
35321,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.19216,1.70297,0.00000,13.058342,0.7,0.0,15.4,3.4,3.741,3.431016
35322,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.22489,1.73255,0.89817,12.375383,3.7,0.6,16.6,7.0,3.591,2.085930
35323,-3.709160,-0.900650,-0.900650,1.114753,5.960128,1.96785,1.83315,0.0,1.83315,11.030100,...,1.14002,1.65512,1.07853,12.095000,1.3,0.0,17.8,8.2,3.306,3.646532


In [14]:
model = ExtraTreesRegressor(n_jobs=-1,random_state=42)
target = 'Eg'
feature_list = [i for i in df.columns if i not in target]
# Execute Genetic Algorithm to obtain Important Feature
from time import time
start = time()
feature_set, optimize_r2_score = ga(df, feature_list, target, 10, 500,model)
feature_set = [feature_list[j] for j in range(len(feature_list)) if feature_set[j]==1]
end = time()
print(f"运行时间T = %0.4f" % (end - start))
print('Optimal Feature Set\n',feature_set,'\nOptimal Accuracy =', round(optimize_r2_score*100), '%')

第 0 次迭代T = 5.0956, optimize_value = 0.8428
第 1 次迭代T = 5.1676, optimize_value = 0.8480
第 2 次迭代T = 5.2075, optimize_value = 0.8480
第 3 次迭代T = 4.8140, optimize_value = 0.8480
第 4 次迭代T = 5.2874, optimize_value = 0.8480
第 5 次迭代T = 4.9485, optimize_value = 0.8480
第 6 次迭代T = 5.0961, optimize_value = 0.8480
第 7 次迭代T = 5.1377, optimize_value = 0.8480
第 8 次迭代T = 5.0380, optimize_value = 0.8480
第 9 次迭代T = 5.1359, optimize_value = 0.8480
第 10 次迭代T = 5.1511, optimize_value = 0.8480
第 11 次迭代T = 4.9505, optimize_value = 0.8480
第 12 次迭代T = 5.0746, optimize_value = 0.8480
第 13 次迭代T = 5.0838, optimize_value = 0.8480
第 14 次迭代T = 5.0548, optimize_value = 0.8480
第 15 次迭代T = 5.1286, optimize_value = 0.8480
第 16 次迭代T = 5.0971, optimize_value = 0.8480
第 17 次迭代T = 5.1366, optimize_value = 0.8480
第 18 次迭代T = 5.0977, optimize_value = 0.8480
第 19 次迭代T = 4.9005, optimize_value = 0.8480
第 20 次迭代T = 5.1217, optimize_value = 0.8480
第 21 次迭代T = 5.0222, optimize_value = 0.8480
第 22 次迭代T = 4.9342, optimize_value = 0.848

第 185 次迭代T = 5.1478, optimize_value = 0.8519
第 186 次迭代T = 5.0517, optimize_value = 0.8519
第 187 次迭代T = 5.0931, optimize_value = 0.8519
第 188 次迭代T = 4.9068, optimize_value = 0.8519
第 189 次迭代T = 5.1595, optimize_value = 0.8519
第 190 次迭代T = 5.0847, optimize_value = 0.8519
第 191 次迭代T = 5.1060, optimize_value = 0.8519
第 192 次迭代T = 5.1389, optimize_value = 0.8519
第 193 次迭代T = 5.1081, optimize_value = 0.8519
第 194 次迭代T = 5.1287, optimize_value = 0.8519
第 195 次迭代T = 5.0141, optimize_value = 0.8519
第 196 次迭代T = 5.0702, optimize_value = 0.8519
第 197 次迭代T = 5.0242, optimize_value = 0.8519
第 198 次迭代T = 5.0650, optimize_value = 0.8519
第 199 次迭代T = 5.0783, optimize_value = 0.8519
第 200 次迭代T = 5.1126, optimize_value = 0.8519
第 201 次迭代T = 4.9500, optimize_value = 0.8519
第 202 次迭代T = 4.9812, optimize_value = 0.8519
第 203 次迭代T = 5.1841, optimize_value = 0.8519
第 204 次迭代T = 4.8534, optimize_value = 0.8519
第 205 次迭代T = 4.9808, optimize_value = 0.8519
第 206 次迭代T = 5.1990, optimize_value = 0.8519
第 207 次迭代T

第 368 次迭代T = 4.9647, optimize_value = 0.8519
第 369 次迭代T = 5.0584, optimize_value = 0.8519
第 370 次迭代T = 5.0039, optimize_value = 0.8519
第 371 次迭代T = 5.1231, optimize_value = 0.8519
第 372 次迭代T = 5.1152, optimize_value = 0.8519
第 373 次迭代T = 5.2064, optimize_value = 0.8519
第 374 次迭代T = 5.0137, optimize_value = 0.8519
第 375 次迭代T = 5.1428, optimize_value = 0.8519
第 376 次迭代T = 4.9676, optimize_value = 0.8519
第 377 次迭代T = 5.0926, optimize_value = 0.8519
第 378 次迭代T = 5.1244, optimize_value = 0.8519
第 379 次迭代T = 5.1416, optimize_value = 0.8519
第 380 次迭代T = 5.1316, optimize_value = 0.8519
第 381 次迭代T = 5.1201, optimize_value = 0.8519
第 382 次迭代T = 4.9993, optimize_value = 0.8519
第 383 次迭代T = 4.9200, optimize_value = 0.8519
第 384 次迭代T = 4.9731, optimize_value = 0.8519
第 385 次迭代T = 4.9706, optimize_value = 0.8519
第 386 次迭代T = 4.8192, optimize_value = 0.8519
第 387 次迭代T = 5.0303, optimize_value = 0.8519
第 388 次迭代T = 5.0219, optimize_value = 0.8519
第 389 次迭代T = 5.1872, optimize_value = 0.8519
第 390 次迭代T

In [16]:
small_features = ABO3_atomic_features[select_atomic_labels]
small_features

,A_atomic_hfomo,A_atomic_hpomo,A_atomic_lfumo,A_atomic_lpumo,A_atomic_ea,A_atomic_ip,A_atomic_ea_by_half_charged_homo,A_atomic_ip_by_half_charged_homo,A_atomic_r_s_neg_1,A_atomic_r_val_neg_1,...,B_atomic_ie_1,B_atomic_ie_2,B_atomic_ndunf,B_atomic_ndval,B_atomic_npunf,B_atomic_npval,B_atomic_nunf,B_atomic_nval,B_atomic_phi,B_atomic_pol
0,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,1.50475,...,6.766510,16.48570,5.0,5.0,0.0,0.0,6.0,6.0,4.65,11.60
1,-4.38669,-2.25963,-2.25963,-2.25963,1.578750,6.198956,1.68613,7.08536,1.84895,0.64805,...,7.881010,17.08400,3.0,7.0,0.0,0.0,3.0,9.0,5.10,7.50
2,-2.01997,-2.01997,-0.47011,-0.47011,0.846499,4.172046,2.68397,6.53600,1.94775,1.37595,...,6.766510,16.48570,5.0,5.0,0.0,0.0,6.0,6.0,4.65,11.60
3,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,6.766510,16.48570,5.0,5.0,0.0,0.0,6.0,6.0,4.65,11.60
4,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,7.902400,16.18770,4.0,6.0,0.0,0.0,4.0,8.0,4.93,8.40
5,-3.50498,-1.78977,-1.78977,-1.78977,-7.811454,-3.641578,2.02672,6.16148,2.14415,1.50475,...,7.881010,17.08400,3.0,7.0,0.0,0.0,3.0,9.0,5.10,7.50
6,-4.19464,-1.78655,-1.78655,-1.78655,1.421494,5.508483,1.47911,5.39953,1.97365,1.10425,...,7.902400,16.18770,4.0,6.0,0.0,0.0,4.0,8.0,4.93,8.40
7,-3.59121,-2.34754,-2.34754,-2.34754,0.757495,5.540776,0.79852,5.67138,2.21125,1.31385,...,6.766510,16.48570,5.0,5.0,0.0,0.0,6.0,6.0,4.65,11.60
8,-3.55283,-1.85729,-1.85729,-1.85729,-2.664785,2.129405,0.60005,6.41133,2.09365,0.34455,...,6.766510,16.48570,5.0,5.0,0.0,0.0,6.0,6.0,4.65,11.60
9,-2.78148,-2.78148,-1.07232,-1.07232,2.903783,6.083156,1.87232,6.37620,2.11275,1.31275,...,6.766510,16.48570,5.0,5.0,0.0,0.0,6.0,6.0,4.65,11.60


In [27]:
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tqdm import tqdm
train_model = ExtraTreesRegressor(n_jobs=-1,random_state=42)
error1 = []
error2 = []
r2_prm = []
error1_base=[]
error2_base=[]
r2_prl=[]
train_set = []
test_set = []
tmp_df = pd.DataFrame()
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_indices = kf.split(X,y)
small_features = small_features[feature_set]
for i,(train_index, test_index) in enumerate(cv_indices):
    train_set.append(train_index)
    test_set.append(test_index)
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y.loc[train_index], y.loc[test_index]
    X_train = X_train[feature_set]
    X_test = X_test[feature_set]
    name = f"fold_{i}"
    train_model.fit(X_train, y_train)
    
    y_pred = train_model.predict(X_test)
    mae_model = mean_absolute_error(y_test, y_pred)
    rmse_model = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_model = r2_score(y_test,y_pred)
    error1.append(mae_model)
    error2.append(rmse_model)
    r2_prm.append(r2_model)
    
    y_base_4 = train_model.predict(small_features)
    tmp_df[f'column_{i}'] = y_base_4
    MAE_BASE_4 = mean_absolute_error(y_base_4,y_eg_abo3)
    RMSE_BASE_4 = np.sqrt(mean_squared_error(y_base_4,y_eg_abo3))
    R2_BASE_4 = r2_score(y_base_4,y_eg_abo3)
    error1_base.append(MAE_BASE_4)
    error2_base.append(RMSE_BASE_4)
    r2_prl.append(R2_BASE_4)
    print(name)
    print(f"Performance on the test set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (mae_model, rmse_model, r2_model))
    print(f"Performance on the small set: MAE = %0.4f,RMSE = %0.4f, R2 = %0.4f" % (MAE_BASE_4, RMSE_BASE_4, R2_BASE_4))
mean_error1 = sum(error1) / len(error1)
mean_error2 = sum(error2) / len(error2)
mean_r2 = sum(r2_prm) / len(r2_prm)
print('Performance on the test set using 5-fold: MAE = %0.4f' % mean_error1)
print('Performance on the test set using 5-fold: RMSE = %0.4f' % mean_error2)
print('Performance on the test set using 5-fold: R2 = %0.4f' % mean_r2)

mean_error1_base = sum(error1_base) / len(error1_base)
mean_error2_base = sum(error2_base) / len(error2_base)
mean_r2_base = sum(r2_prl) / len(r2_prl)
print('Performance on the valid set using 5-fold: MAE = %0.4f' % mean_error1_base)
print('Performance on the valid set using 5-fold: RMSE = %0.4f' % mean_error2_base)
print('Performance on the valid set using 5-fold: R2 = %0.4f' % mean_r2_base)

fold_0
Performance on the test set: MAE = 0.3507,RMSE = 0.4779, R2 = 0.8511
Performance on the small set: MAE = 0.3914,RMSE = 0.5180, R2 = 0.6475
fold_1
Performance on the test set: MAE = 0.3547,RMSE = 0.4853, R2 = 0.8453
Performance on the small set: MAE = 0.3868,RMSE = 0.4928, R2 = 0.7014
fold_2
Performance on the test set: MAE = 0.3548,RMSE = 0.4855, R2 = 0.8503
Performance on the small set: MAE = 0.4091,RMSE = 0.5415, R2 = 0.6390
fold_3
Performance on the test set: MAE = 0.3483,RMSE = 0.4784, R2 = 0.8488
Performance on the small set: MAE = 0.4050,RMSE = 0.5308, R2 = 0.6532
fold_4
Performance on the test set: MAE = 0.3464,RMSE = 0.4753, R2 = 0.8515
Performance on the small set: MAE = 0.3895,RMSE = 0.5013, R2 = 0.6851
Performance on the test set using 5-fold: MAE = 0.3510
Performance on the test set using 5-fold: RMSE = 0.4805
Performance on the test set using 5-fold: R2 = 0.8494
Performance on the valid set using 5-fold: MAE = 0.3964
Performance on the valid set using 5-fold: RMSE =

In [18]:
best_train_index = train_set[4]
best_test_index = test_set[4]
best_X_train = X.loc[best_train_index].reset_index(drop=True)
best_X_test = X.loc[best_test_index].reset_index(drop=True)
best_y_train = y.loc[best_train_index].reset_index(drop=True)
best_y_test = y.loc[best_test_index].reset_index(drop=True)

In [19]:
best_X_train = best_X_train[feature_set]
best_X_test = best_X_test[feature_set]

In [38]:
model = ExtraTreesRegressor(n_jobs=-1, random_state=42)
model.fit(best_X_train, best_y_train)
y_pred = model.predict(best_X_test)
MAE_4 = mean_absolute_error(best_y_test,y_pred)
RMSE_4 = np.sqrt(mean_squared_error(best_y_test, y_pred))
R2_4 = r2_score(best_y_test,y_pred)
print('Performance on the test set: MAE = %0.4f' % MAE_4)
print('Performance on the test set: RMSE = %0.4f' % RMSE_4)
print('Performance on the test set: R2 SCORE = %0.4f' % R2_4)

Performance on the test set: MAE = 0.3464
Performance on the test set: RMSE = 0.4753
Performance on the test set: R2 SCORE = 0.8515


In [28]:
# import json
# with open("best_feature_ec.json", "w") as f:
#     json.dump(feature_set, f)

In [37]:
# import pickle
# with open('model_ec.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [39]:
error_data = pd.DataFrame(columns = ['formula','valid_Eg','pred_Eg'])
error_data['formula'] = small_data['formula']
error_data['valid_Eg'] = small_data['target']
error_data['pred_Eg'] = tmp_df['column_4']
error_data['diff'] = abs(error_data['valid_Eg'] - error_data['pred_Eg'])
error_data = error_data.sort_values(by='diff', ascending=False).reset_index(drop = True)
error_data

,formula,valid_Eg,pred_Eg,diff
0,BaCeO3,3.941353,2.622542,1.318811
1,TaAgO3,3.135210,2.150132,0.985078
2,TiPbO3,2.584444,1.679295,0.905149
3,DyInO3,2.996601,3.884960,0.888358
4,ZrPbO3,3.477219,2.598152,0.879067
5,NbAgO3,2.606289,1.735743,0.870546
6,InBiO3,1.050888,1.886769,0.835881
7,HfPbO3,3.538460,2.738099,0.800361
8,InSbO3,0.751440,1.537170,0.785730
9,PmInO3,3.056738,3.730180,0.673441


In [23]:
# error_data.to_csv('EC_errror.csv')

In [24]:
train_data = X[feature_set]
predict_y = model.predict(train_data)

In [25]:
large_error_data = pd.DataFrame(columns = ['valid_Eg','pred_Eg'])
large_error_data['valid_Eg'] = y
large_error_data['pred_Eg'] = predict_y
large_error_data['diff'] = abs(large_error_data['valid_Eg'] - large_error_data['pred_Eg'])
large_error_data = large_error_data.sort_values(by='diff', ascending=False).reset_index(drop = True)
large_error_data

,valid_Eg,pred_Eg,diff
0,1.357424,5.538459,4.181036
1,0.963888,4.652260,3.688372
2,1.091527,4.770885,3.679358
3,1.230096,4.278437,3.048341
4,1.329234,4.238745,2.909511
...,...,...,...
35320,4.528787,4.528787,0.000000
35321,2.312129,2.312129,0.000000
35322,1.844291,1.844291,0.000000
35323,1.277718,1.277718,0.000000


In [26]:
large_error_data.to_csv('large_ec_error.csv')

In [41]:
ec_error_big = pd.DataFrame(columns = ['Eg_error'])
ec_error_big['Eg_error'] = abs(best_y_test-y_pred)
ec_error_big

,Eg_error
0,0.090501
1,0.000400
2,0.035522
3,0.017994
4,0.430191
...,...
7060,0.024589
7061,0.023237
7062,0.049489
7063,0.214915


In [42]:
# ec_error_big.to_csv('ec_error_big.csv')